# Analisis Distribusi dan Kebutuhan Satuan Pendidikan di Kabupaten Bone
Notebook ini memuat pipeline lengkap:
- Load data Excel
- Preprocessing
- Modeling (beberapa algoritma)
- Evaluasi
- Deployment sederhana dengan Gradio

In [22]:
# Kalau pertama kali, jalankan ini sekali (boleh dihapus setelah OK):
# !pip install pandas numpy scikit-learn gradio openpyxl

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

import gradio as gr
from IPython.display import display

# Path file Excel (sama persis seperti di folder)
FILE_PATH = "Data Induk Satuan Pendidikan  - JUMLAH Kabupaten 190700 - ASC - 24 November 2025.xlsx"


In [23]:
# Baca Excel tanpa header karena header sebenarnya ada di baris ke-4 (index 3)
df_raw = pd.read_excel(FILE_PATH, header=None)
print("=== 5 BARIS PERTAMA RAW ===")
display(df_raw.iloc[:8])

# Ambil baris ke-4 (index 3) sebagai header
header_row = df_raw.iloc[3]
print("\n=== HEADER YANG DIPAKAI ===")
print(header_row.values)

# Data mulai baris ke-5 (index 4)
df = df_raw.iloc[4:].copy()
df.columns = header_row

# Buang baris yang 'Kode Wilayah' kosong (jaga-jaga)
df = df[df["Kode Wilayah"].notna()].reset_index(drop=True)

print("\n=== DATASET BERSIH (SEBELUM KONVERSI NUMERIK) ===")
print("Shape:", df.shape)
display(df.head())


=== 5 BARIS PERTAMA RAW ===


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Jumlah Satuan Pendidikan - KAB. BONE - 06122025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Wilayah:,Semua Kecamatan di KAB. BONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kode Wilayah,Nama Wilayah,TK Sederajat,KB Sederajat,TPA,SPS,SD Sederajat,SMP Sederajat,SMA Sederajat,SMK Sederajat,SLB,Dikmas,Total
4,190716,KEC. ULAWENG,15,0,0,0,28,6,3,0,0,2,54
5,190705,KEC. TONRA,16,3,0,2,16,5,2,0,0,1,45
6,190721,KEC. AJANGALE,19,0,0,0,32,8,3,1,0,1,64
7,190727,KEC. TELLU LIMPOE,3,0,0,0,14,6,1,0,0,0,24



=== HEADER YANG DIPAKAI ===
['Kode Wilayah' 'Nama Wilayah' 'TK Sederajat' 'KB Sederajat' 'TPA' 'SPS'
 'SD Sederajat' 'SMP Sederajat' 'SMA Sederajat' 'SMK Sederajat' 'SLB'
 'Dikmas' 'Total']

=== DATASET BERSIH (SEBELUM KONVERSI NUMERIK) ===
Shape: (29, 13)


3,Kode Wilayah,Nama Wilayah,TK Sederajat,KB Sederajat,TPA,SPS,SD Sederajat,SMP Sederajat,SMA Sederajat,SMK Sederajat,SLB,Dikmas,Total
0,190716,KEC. ULAWENG,15,0,0,0,28,6,3,0,0,2,54
1,190705,KEC. TONRA,16,3,0,2,16,5,2,0,0,1,45
2,190721,KEC. AJANGALE,19,0,0,0,32,8,3,1,0,1,64
3,190727,KEC. TELLU LIMPOE,3,0,0,0,14,6,1,0,0,0,24
4,190701,KEC. BONTOCANI,14,0,0,0,24,9,3,0,0,1,51


In [24]:
# Kolom kategori
COL_KODE = "Kode Wilayah"
COL_NAMA = "Nama Wilayah"

# Kolom numerik (dari struktur file)
NUM_COLS = [
    "TK Sederajat",
    "KB Sederajat",
    "TPA",
    "SPS",
    "SD Sederajat",
    "SMP Sederajat",
    "SMA Sederajat",
    "SMK Sederajat",
    "SLB",
    "Dikmas",
    "Total"
]

# Konversi ke numerik (kalau ada string/NaN jadi angka)
for col in NUM_COLS:
    df[col] = pd.to_numeric(df[col], errors="coerce")

print("=== TIPE DATA SETELAH KONVERSI NUMERIK ===")
print(df.dtypes)

display(df.head())

# Fitur yang dipakai untuk model (bisa diubah, tapi ini sudah logis)
FEATURE_COLS = [
    "TK Sederajat",
    "KB Sederajat",
    "TPA",
    "SPS",
    "SD Sederajat",
    "SMP Sederajat",
    "SMA Sederajat",
    "SMK Sederajat",
]

COL_TOTAL = "Total"

=== TIPE DATA SETELAH KONVERSI NUMERIK ===
3
Kode Wilayah      object
Nama Wilayah      object
TK Sederajat     float64
KB Sederajat     float64
TPA              float64
SPS              float64
SD Sederajat     float64
SMP Sederajat    float64
SMA Sederajat    float64
SMK Sederajat    float64
SLB              float64
Dikmas           float64
Total            float64
dtype: object


3,Kode Wilayah,Nama Wilayah,TK Sederajat,KB Sederajat,TPA,SPS,SD Sederajat,SMP Sederajat,SMA Sederajat,SMK Sederajat,SLB,Dikmas,Total
0,190716,KEC. ULAWENG,15.0,0.0,0.0,0.0,28.0,6.0,3.0,0.0,0.0,2.0,54.0
1,190705,KEC. TONRA,16.0,3.0,0.0,2.0,16.0,5.0,2.0,0.0,0.0,1.0,45.0
2,190721,KEC. AJANGALE,19.0,0.0,0.0,0.0,32.0,8.0,3.0,1.0,0.0,1.0,64.0
3,190727,KEC. TELLU LIMPOE,3.0,0.0,0.0,0.0,14.0,6.0,1.0,0.0,0.0,0.0,24.0
4,190701,KEC. BONTOCANI,14.0,0.0,0.0,0.0,24.0,9.0,3.0,0.0,0.0,1.0,51.0


In [25]:
# Imputasi nilai kosong dengan median (untuk jaga-jaga)
for col in FEATURE_COLS + [COL_TOTAL]:
    df[col] = df[col].fillna(df[col].median())

print("=== CEK MISSING VALUE ===")
print(df[FEATURE_COLS + [COL_TOTAL]].isna().sum())

# Buat kelompok berdasarkan Total sekolah → Sedikit, Sedang, Banyak
df["total_group"] = pd.qcut(
    df[COL_TOTAL],
    q=3,
    labels=["Sedikit", "Sedang", "Banyak"]
)

mapping_prioritas = {
    "Sedikit": "Prioritas Tinggi",
    "Sedang": "Prioritas Sedang",
    "Banyak": "Prioritas Rendah",
}

df["Prioritas_Label"] = df["total_group"].map(mapping_prioritas)

print("\n=== DISTRIBUSI LABEL PRIORITAS ===")
print(df["Prioritas_Label"].value_counts())
display(df[[COL_NAMA, COL_TOTAL, "total_group", "Prioritas_Label"]].head())


=== CEK MISSING VALUE ===
3
TK Sederajat     0
KB Sederajat     0
TPA              0
SPS              0
SD Sederajat     0
SMP Sederajat    0
SMA Sederajat    0
SMK Sederajat    0
Total            0
dtype: int64

=== DISTRIBUSI LABEL PRIORITAS ===
Prioritas_Label
Prioritas Tinggi    10
Prioritas Rendah    10
Prioritas Sedang     9
Name: count, dtype: int64


3,Nama Wilayah,Total,total_group,Prioritas_Label
0,KEC. ULAWENG,54.0,Sedikit,Prioritas Tinggi
1,KEC. TONRA,45.0,Sedikit,Prioritas Tinggi
2,KEC. AJANGALE,64.0,Sedang,Prioritas Sedang
3,KEC. TELLU LIMPOE,24.0,Sedikit,Prioritas Tinggi
4,KEC. BONTOCANI,51.0,Sedikit,Prioritas Tinggi


In [26]:
X = df[FEATURE_COLS].values
y_label = df["Prioritas_Label"].values

le = LabelEncoder()
y = le.fit_transform(y_label)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("=== INFO DATA TRAIN/TEST ===")
print("X_train:", X_train.shape)
print("X_test :", X_test.shape)
print("Kelas  :", le.classes_)


=== INFO DATA TRAIN/TEST ===
X_train: (23, 8)
X_test : (6, 8)
Kelas  : ['Prioritas Rendah' 'Prioritas Sedang' 'Prioritas Tinggi']


In [27]:
models = {}

models["LogisticRegression"] = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000, random_state=42))
])

models["KNN"] = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", KNeighborsClassifier(n_neighbors=5))
])

models["GaussianNB"] = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", GaussianNB())
])

models["DecisionTree"] = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", DecisionTreeClassifier(random_state=42))
])

models["RandomForest"] = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", RandomForestClassifier(
        n_estimators=200,
        random_state=42
    ))
])

best_score = -np.inf
best_name = None
best_model = None

cv = KFold(n_splits=5, shuffle=True, random_state=42)

print("=== HASIL CROSS-VALIDATION (F1-MACRO) ===")
for name, model in models.items():
    scores = cross_val_score(
        model, X_train, y_train,
        cv=cv, scoring="f1_macro"
    )
    print(f"{name:15s} | mean={scores.mean():.4f} | std={scores.std():.4f}")

    if scores.mean() > best_score:
        best_score = scores.mean()
        best_name = name
        best_model = model

print(f"\nModel terbaik berdasarkan F1-macro: {best_name} ({best_score:.4f})")


=== HASIL CROSS-VALIDATION (F1-MACRO) ===
LogisticRegression | mean=0.8144 | std=0.1698
KNN             | mean=0.5879 | std=0.2823
GaussianNB      | mean=0.5733 | std=0.3135
DecisionTree    | mean=0.6733 | std=0.2502
RandomForest    | mean=0.8489 | std=0.1477

Model terbaik berdasarkan F1-macro: RandomForest (0.8489)


In [28]:
# Latih model terbaik pada seluruh training data
best_model.fit(X_train, y_train)

# Prediksi test
y_pred = best_model.predict(X_test)

print("=== EVALUASI DI TEST SET ===")
print("Accuracy:", accuracy_score(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


=== EVALUASI DI TEST SET ===
Accuracy: 0.8333333333333334

Classification Report:
                  precision    recall  f1-score   support

Prioritas Rendah       1.00      1.00      1.00         2
Prioritas Sedang       1.00      0.50      0.67         2
Prioritas Tinggi       0.67      1.00      0.80         2

        accuracy                           0.83         6
       macro avg       0.89      0.83      0.82         6
    weighted avg       0.89      0.83      0.82         6

Confusion Matrix:
[[2 0 0]
 [0 1 1]
 [0 0 2]]


In [29]:
def predict_priority(tk, kb, tpa, sps, sd, smp, sma, smk):
    X_new = pd.DataFrame([[
        tk, kb, tpa, sps, sd, smp, sma, smk
    ]], columns=FEATURE_COLS)

    y_int = best_model.predict(X_new)[0]
    y_label = le.inverse_transform([y_int])[0]

    return f"Kategori Kebutuhan Sekolah: {y_label}"


app = gr.Interface(
    fn=predict_priority,
    inputs=[
        gr.Number(label="TK Sederajat"),
        gr.Number(label="KB Sederajat"),
        gr.Number(label="TPA"),
        gr.Number(label="SPS"),
        gr.Number(label="SD Sederajat"),
        gr.Number(label="SMP Sederajat"),
        gr.Number(label="SMA Sederajat"),
        gr.Number(label="SMK Sederajat"),
    ],
    outputs=gr.Textbox(label="Hasil Prediksi"),
    title="Prediksi Kebutuhan Satuan Pendidikan - Kabupaten Bone",
    description="Masukkan jumlah satuan pendidikan per jenjang untuk menentukan kategori prioritas (Tinggi/Sedang/Rendah)."
)

app.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
